In [ ]:
# Python


# Happy Quotes - the quotes that brighten your day

This script is split into three main sections:

1 - A BeautifulSoup based script that scrappes the website "https://quotes.toscrape.com" to extract all the quotes and all the authors.

2 - A script that calls the API endpoint: "https://jsonplaceholder.typicode.com/comments?postId=id" to extract "made up" comments for each one of our quotes.

3 - A script that transforms all the data extracted in the previous sections and then loads the data into a database.  
 

# Section 1
Let's start with Beautiful Soup and extract all the data from "https://quotes.toscrape.com" about the quotes and the authors.

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
# URL Constants
BASE_URL = "https://quotes.toscrape.com"
START_URL = BASE_URL + "/page/1/"

In [ ]:
# BS4 Locators for the quotes
QUOTE_LOCATOR = "div.quote"
TEXT_LOCATOR = "span.text"
AUTHOR_LOCATOR = "small.author"
TAGS_LOCATOR = "div.tags a.tag"
AUTHOR_LINK_LOCATOR = "div.quote span a"
NEXT_PAGE_LOCATOR = "li.next a"

In [ ]:
# BS4 Locators for the author details
AUTHOR_BORN_DATE_LOCATOR = "span.author-born-date"
AUTHOR_BORN_LOCATION_LOCATOR = "span.author-born-location"
AUTHOR_DESCRIPTION_LOCATOR = "div.author-description"

In [ ]:
# Start with empty variables
# global variables:
all_quotes = []
authors_urls = set()    # to store unique urls to each author's page
all_tags = set()    # to store unique tags
all_authors_names = set()    # to store unique authors' names

In [ ]:
# helper function

def maxSizeOfKey(data_list, key_name):
    max_size = 0
    for element in data_list:
        if len(element[key_name]) > max_size:
            max_size = len(element[key_name])
    if max_size > 255:
        print(f"Max size of {key_name} is {max_size} characters. Requires a TEXT data type in the database.")
    else:
        print(f"Max size of {key_name} is {max_size} characters. Requires a VARCHAR data type in the database.")

def printAll(data_list, data_name):
    print(f"\nAll the {data_name}:")
    
    for element in data_list:
        if isinstance(element, str):
            print(element, end=', ')
        else:
            print(element)
        
    print(f"\nTotal {data_name} = {len(data_list)}")

# Function to extract quotes and update the set of author URLs
# the variable quotes is passed by reference, it i updated on each get request

def getQuotes(soup, quotes):
    
    global all_tags, all_authors, authors_urls

    # get all the quotes from the current page
    quote_elements = soup.select(QUOTE_LOCATOR)
    
    # go through each single quote in the current page
    for quote_element in quote_elements:
        text = quote_element.select_one(TEXT_LOCATOR).text
        author = quote_element.select_one(AUTHOR_LOCATOR).text
        tags = [tag.text for tag in quote_element.select(TAGS_LOCATOR)]
        author_link = BASE_URL + quote_element.select_one(AUTHOR_LINK_LOCATOR).attrs['href']
        
        # update, by reference, the global variable, all_quotes, which is a list of dicts
        quotes.append({
            'text': text,
            'author': author,
            'tags': tags
        })
        
        # update the global variable, all_tags, which is a set of unique elements
        for tag in tags:
            all_tags.add(tag)
        
        # update the global variable, all_authors_names, which is a set of unique elements
        all_authors_names.add(author)
        
        # update the global variable, authors_urls, which is a set of unique elements
        authors_urls.add(author_link)

# Function to extract the details of each author 
def getAuthors():
    
    global authors_urls
    # start with an empty list
    authors = []
    
    # go through each of authors_urls and get their details
    for author_url in authors_urls:
        page = requests.get(author_url)

        # add error message if request fails

        soup = BeautifulSoup(page.content, 'html.parser')
        
        # get the details of each author
        author_name = soup.select_one("h3.author-title").text.strip()
        born_date = soup.select_one(AUTHOR_BORN_DATE_LOCATOR).text
        born_location = soup.select_one(AUTHOR_BORN_LOCATION_LOCATOR).text
        description = soup.select_one(AUTHOR_DESCRIPTION_LOCATOR).text.strip()
        
        # create a list of dicts
        authors.append({
            'name': author_name,
            'born_date': born_date,
            'born_location': born_location,
            'description': description
        })
    
    # return a list of dicts, with the details of all the authors
    return authors

In [ ]:
# Main scraping logic
page_url = START_URL

# first, get all the quotes
while page_url:     # stops when page_url = None
    page = requests.get(page_url)
    
    # add error message if request fails
    
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # global variables are passed by reference, all_quotes is updated on each get request
    getQuotes(soup, all_quotes)
    
    # get next page url
    next_page_element = soup.select_one(NEXT_PAGE_LOCATOR)  
    page_url = BASE_URL + next_page_element.attrs['href'] if next_page_element else None    # None -> no more pages

# next, let's get all the authors
all_authors = getAuthors()


# Python variable - all_quotes

![all_quotes](img\var_all_quotes.png)


In [ ]:
# Print the final result
printAll(all_quotes, 'quotes')

# check which data type we need to use in the database for key 'text'
print("Which database data type do we need for quote['text']?")
maxSizeOfKey(all_quotes, 'text')

# Python variable - all_authors
![all_authors](img\var_all_authors.png)

In [ ]:
# Print the final result
printAll(all_authors, 'authors')

# check which data type we need to use in the database for key 'text'
print("Which database data type do we need for author['description']?")
maxSizeOfKey(all_authors, 'description')

In [ ]:
# check the variables: all_tags and all_authors_names
# convert the sets into ordered lists and check the required data type for the database

all_tags = sorted(all_tags) # a list of all the tags
printAll(all_tags, 'tags')
print("Which database data type do we need for 'all_tags'?", end=' ')
print("Use TEXT data type in the database." if sum(len(tag) for tag in all_tags) > 255 else "Use VARCHAR data type in the database.")
print("Total lenght = ",sum(len(tag) for tag in all_tags))

all_authors_names = sorted(all_authors_names) # a list of all the authors' names
printAll(all_authors_names, 'authors_names')
print("Which database data type do we need for 'all_authors_names'?", end=' ')
print("Use TEXT data type in the database." if sum(len(author) for author in all_authors_names) > 255 else "Use VARCHAR data type in the database.")
print("Total lenght = ", sum(len(author) for author in all_authors_names))

The variables "all_quotes" and "all_authors" contain all the data that we extracted from the "https://quotes.toscrape.com" website.  
The variables "all_tags" and "all_authors_names" contain lists with metadata, the unique values of each tag and each author's name.

# Section 1 - ends here

# Section 2
We'll now extract 100 comments (to match the 100 quotes) from the API endpoint: 
"https://jsonplaceholder.typicode.com/comments?postId=id"

In [ ]:
# Configure the API settings

# import requests was completed at the start

# 1 - define the API endpoint
endpoint = "https://jsonplaceholder.typicode.com/comments"

# 2 - define the parameters
parameters = {
    'postId' : None
}

In [ ]:
# API functions

def fetchPostComments(post_id):
    
    # API endpoint updated with post_id params
    global endpoint, parameters
    parameters['postId'] = post_id

    # 3 - call the API with updated params
    response = requests.get(url=endpoint, params=parameters)
    
    # testing error messages
    # if post_id in (1, 10, 20, 40, 50, 60, 70, 80, 90, 100):
    #     print(f"ERROR: Failed to fetch comments for post ID {post_id}")
    #     return [f"ERROR: Failed to fetch comments for post ID {post_id}"]
    
    if response.status_code == 200:
        # 4 - return the response from the API
        return response.json()
    else:
        print(f"ERROR: Failed to fetch comments for post ID {post_id}")
        return [f"ERROR: Failed to fetch comments for post ID {post_id}"]

In [ ]:

all_comments = []

# total quotes fetched = len(all_quotes) = 100
for post_id in range(1, len(all_quotes) + 1):
    comments = fetchPostComments(post_id)
    all_comments.extend(comments)

print(f"Total number of comments fetched: {len(all_comments)}")

# Python variable - all_comments
![all_comments](img\var_all_comments.png)

In [ ]:
# print all_comments
printAll(all_comments, 'comments')

# check which database data type is required for key 'body'
maxSizeOfKey(all_comments, 'body')

The variable "all_comments" includes all the comments extracted from the API endpoint, that's 5 comments for each one of 100 quotes, so that's a total of 500 comments.

This completes the "Extract" phase of the script.

# Section 2 - ends here

# Section 3
This section of the script Transforms the extracted data and Loads it into a SQL database.
  
1. Use dotenv to import the db access details and the name of the SQL init script
2. Create a database from the SQL init script
3. Import a Python module with the Pydantic based classes to interact with the database
4. Clean up the data, Transform the variables being used, Check the data types, Map the data to Python classes:
    1. all_quotes = contains all the  details about each quote 
    2. all_authors = contains all the  details about the authors
    3. all_comments = contains all the comments for each quote
5. Use the Pydantic classes to load the data into the SQL database

<p style="color:red; font-size:20px;"><strong>Beware! Update the settings in the file .env to match your environment!</strong></p>



In [ ]:
# import the database and models classes

from happy_models import *
from datetime import datetime
from dotenv import load_dotenv
import os


In [ ]:
# Test connection to the database server
# Beware! .env default settings are: host = localhost, user = root, pwd = '', port = 3307
# You must update the .env file with the settings for your environment!

# set the name for your database
# we do that here, as MySQLDB() class allows one class instance for each database
# Load the environment variables from the .env file
load_dotenv('.env')
DB_NAME = os.getenv('DATABASE_NAME')
# create an object of MySQLDB class to manage your database
db = MySQLDB(DB_NAME)
db.test_connection()

# Let's create these tables in the database
![database schema](img\database_schema.png)


In [ ]:
# run this once to create the database and the tables
# if the database and tables already exist, then they are not overwritten

# during testing, keep overwrite = True, to start from a new DB on each test run
db.create_database(True)

author_sql ="""
-- Create the author table
CREATE TABLE author (
    id_author INT AUTO_INCREMENT PRIMARY KEY,
    date_created TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    date_modified TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    name VARCHAR(255) NOT NULL,
    birth_date DATE,
    birth_city VARCHAR(255),
    birth_state VARCHAR(255),
    birth_country VARCHAR(255),
    description TEXT
);
"""
db.create_table('author', author_sql, False)

quote_sql="""
-- Create the quote table
CREATE TABLE quote (
    id_quote INT AUTO_INCREMENT PRIMARY KEY,
    date_created TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    date_modified TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    content TEXT NOT NULL,
    author_id INT NOT NULL,
    tags VARCHAR(255),
    FOREIGN KEY (author_id) REFERENCES author(id_author)
);
"""
db.create_table('quote', quote_sql, False)

comment_sql="""
-- Create the comment table
CREATE TABLE comment (
    id_comment INT AUTO_INCREMENT PRIMARY KEY,
    date_created TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    date_modified TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    quote_id INT NOT NULL,
    title VARCHAR(255) NOT NULL,
    details TEXT,
    user_email VARCHAR(255),
    FOREIGN KEY (quote_id) REFERENCES quote(id_quote)
);
"""
db.create_table('comment', comment_sql, False)

metadata_sql="""
-- Create the metadata table
CREATE TABLE metadata (
    id_key INT AUTO_INCREMENT PRIMARY KEY,
    date_created TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    date_modified TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    key_name VARCHAR(255) NOT NULL,
    key_value TEXT NOT NULL
);
"""
db.create_table('metadata', metadata_sql, False)

# Let's clean up and transform the data
We'll be mapping the Python variables to Pydantic classes and then save the data in a SQL database.
![Map of the Pydantic Classes](img\oop_classes.png)


In [ ]:
# let's start with the 'author' table, because it doesn't have any Foreign Keys

for author in all_authors:
    name = author['name']
    dob = datetime.strptime(author['born_date'], "%B %d, %Y")   # convert string to date
    description = author['description']
    born_location = author['born_location'][2:]     # strips the characters 'in ' from the start of the string
    location_list = born_location.split(', ')       # creates a list of places

    match len(location_list):       # allocate the right place to the right location
        case 0:
            birth_city = None
            birth_state = None
            birth_country = None
        case 1:
            birth_city = None
            birth_state = None
            birth_country = location_list[0]
        case 2:
            birth_city = location_list[0]
            birth_state = None
            birth_country = location_list[1]
        case 3:
            birth_city = location_list[0]
            birth_state = location_list[1]
            birth_country = location_list[2]
        case _:
            birth_city = location_list[0]
            birth_state = location_list[1]
            birth_country = location_list[2]
    
    # leverage Pydantic type annotations and validations to do type checking and coercion if needed
    author_obj = Author(
        name = name,
        birth_date = dob,
        birth_city = birth_city,
        birth_state = birth_state,
        birth_country = birth_country,
        description = description
    )

    # save each author to the database
    author_obj.db_save(db)

    # print each object to confirm that an id was assigned to it
    print(author_obj)

In [ ]:
# Cleaning the data. 
# The 'quote' table, has one Foreign Key from 'author', so let's check that quote['author'] == author['name'].
# If there are any misspellings, then we correct quote['author']

# List with all author names extracted from the quotes
print("Total names_from_quotes: ", len(all_authors_names))

# List with all author names extracted from the author page
names_from_authors = [author['name'] for author in all_authors]
print("Total names_from_authors: ", len(names_from_authors))

# Let's find the differences between the two lists

# Convert lists to sets
set_quotes = set(all_authors_names)
set_authors = set(names_from_authors)

# Find names in quotes but not in authors
names_in_quotes_not_in_authors = set_quotes - set_authors

# Find names in authors but not in quotes
names_in_authors_not_in_quotes = set_authors - set_quotes

# Print the results
print("Names in quotes but not in authors:", names_in_quotes_not_in_authors)
print("Names in authors but not in quotes:", names_in_authors_not_in_quotes)


In [ ]:
# Cleaning the data. 
# Let's replace all occurences of 'Alexandre Dumas fils' in quotes with 'Alexandre Dumas-fils'
# Beware! If you've run this script before, then you will no longer find these differences in the data.

i = 0
for quote in all_quotes:
    if quote['author'] == 'Alexandre Dumas fils':
        quote['author'] = 'Alexandre Dumas-fils'
        i += 1
print(f"Completed {i} replacements")

# update list of all_authors_names, it will be saved in the database later
all_authors_names = sorted(names_from_authors)
print("Total authors' names: ", len(all_authors_names))
print(all_authors_names)


In [ ]:
# the 'quote' table, has one Foreign Key from 'author'

for quote in all_quotes:
    
    # Slicing to remove the first and last characters, they're unicode codes that we don't want
    content = quote['text'][1:-1]     
        
    tags = quote['tags']

    # fetch author_id from the database
    sql_query = """
    SELECT id_author FROM author WHERE name = %s
"""
    author_name = quote['author']
    author_id = db.sql_query(sql_query, (author_name, ))[0]['id_author']
    
    quote_obj = Quote(
        content = content,
        author_id = author_id,
        tags = tags
    )

    quote_obj.db_save(db)
    print(quote_obj)

In [ ]:
# The 'comment' table, has one Foreign Key from 'quote'
# Let's check that comment.quote_id <= Max(quote.id_quote)

sql_query = """SELECT MAX(id_quote) AS max_id_quote
FROM quote"""
max_quote_id = db.sql_query(sql_query, ())[0]['max_id_quote']

# Initialize the flag
error_found = False
for comment in all_comments:
    if comment['postId'] <= max_quote_id:
        pass
    else:
        print("Error in comment with postId = ", comment['postId'])
        error_found = True
if not error_found:
    print("No errors found in comments")



In [ ]:
# Let's save all the comments in the database with the Foreign Key from 'quote'

for comment in all_comments:
    quote_id = comment['postId']
    title = comment['name']
    details = comment['body']
    user_email = comment['email']

    # instantiate the comment object
    comment_obj = Comment(
        quote_id = quote_id,
        title = title,
        details = details,
        user_email = user_email
    )

    # save the comment object to the database
    comment_obj.db_save(db)
    print(comment_obj)

In [ ]:
# Lastly, let's save the Metadata, it holds some details that we can use later

metadata_tags = Metadata(
    key_name = 'all_tags',
    key_value = all_tags
)

metadata_tags.db_save(db)
print(metadata_tags)

metadata_authors = Metadata(
    key_name = 'all_authors',
    key_value = all_authors_names
)

metadata_authors.db_save(db)
print(metadata_authors)



The script ends here.  
We've used BeautifulSoup4 to webscrape one website, then consummed one API to get mock-data for our comments.  
We've used the Pydantic library to leverage its type annotations and validation features before loading the data to a SQL database.  
The configuration settings for the SQL database are stored in a .env file, for added security.  

We now have a database,"happy_quotes", that contains four tables:  
1. "author" - with all the details about each author
2. "quote" - with all the details about each quote and a Foreign Key into the "author" table
3. "comment" - with all the details about each comment and a Foreign Key into the "quote" table
4. "metadata" - with two lists, 'all_tags' and 'all_authors'
  
This completes the "Transform" and "Load" phases of the script.  
  
# Section 3 - ends here